In [ ]:
import pandas_datareader as web
import matplotlib.pyplot as plt
import pandas as pd
from dask.distributed import Client
from dask.distributed import as_completed
import pandas_market_calendars as mcal
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, RepeatedKFold, train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import matplotlib
from scipy import stats

from dask.distributed import Client
from dask.distributed import as_completed
from IPython.display import clear_output
import time 
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm
from sklearn.model_selection import TimeSeriesSplit

from statsmodels.tsa.statespace.sarimax import SARIMAX

import sys
#if not sys.warnoptions:
import warnings
    
from dask.distributed import Client
from dask.distributed import as_completed

pd.set_option("max_rows", 10)

start = "1970-01-01"

In [ ]:
#sp500_ancient = web.DataReader("^GSPC", "yahoo",start=start)
sp500 = web.DataReader("^GSPC", "yahoo",start=start)
vix = web.DataReader("VIXCLS","fred",start=start)
T10Y3M = web.DataReader("T10Y3M","fred",start=start)
FEDFUNDS = web.DataReader("FEDFUNDS","fred",start=start)


In [ ]:
#from sklearn.model_selection import TimeSeriesSplit

In [ ]:
data_ = sp500[['Adj Close']].asfreq('Q',method='Pad')

#cv_outer = KFold(n_splits=k_, shuffle=False)
#cv_outer = train_test_split(data_, test_size=0.3, random_state=0,shuffle=False,)

k_=3
#tscv_outer = TimeSeriesSplit(n_splits=k_,max_train_size=int(len(data_)/k_),test_size=int(len(data_)/k_))
tscv_outer = TimeSeriesSplit(n_splits=k_)

outer_results = []
outer_models = []

outer_sets = []

for fold, (train_index, test_index) in enumerate(tscv_outer.split(data_)):
    print("Fold: {}".format(fold))
    print("TRAIN indices:", train_index, "\n", "TEST indices:", test_index)
    print("\n")
    X_train, X_test = data_.iloc[train_index], data_.iloc[test_index]
    outer_sets.append([X_train, X_test])
    #y_train, y_test = y[train_index], y[test_index]

"""
#for trainv_ix, test_ix in cv_outer.split(data_):
#for trainv_ix, test_ix in cv_outer.split(data_):

#data_index = ts_cv_kfolds(data_.index,k_,True,.75)

#for di in range(0,len(data_index[0]),1):
for trainv_ix, test_ix in tscv_outer.split(data_):
    #trainv_ix = data_index[0][di]
    #test_ix = data_index[1][di]
    
    # split data
    #for i in range(0,len(data_index[0])):
        #print(i)
    X_trainv, X_test = data_.iloc[trainv_ix], data_.iloc[test_ix]
    outer_sets.append([X_trainv, X_test])
    #y_trainv, y_test = y.iloc[trainv_ix], y.iloc[test_ix]
"""

In [ ]:
len(outer_sets)

In [ ]:
[len(o[0]) for o in outer_sets]

In [ ]:
[len(o[1]) for o in outer_sets]

In [ ]:
#[o[0] for o in outer_sets]

In [ ]:
def inner_arima(test_sets,model_=False):
    
    k_=2
    
    #cv_inner = KFold(n_splits=k_, shuffle=False)
    #tscv_inner = TimeSeriesSplit(n_splits=k_,max_train_size=int(len(test_sets[0])/k_),test_size=int(len(test_sets[0])/k_))    
    tscv_inner = TimeSeriesSplit(n_splits=k_)
    
    X_trainv = test_sets[0]
    #print(X_trainv)
    X_test = test_sets[1]
    #print(X_test)
    #print(X_test.index)
    
    inner_results = list()
    inner_models = list()

    #for trainv_ix, test_ix in ts_cv_kfolds(data_.index,k_):
    
    # split data
    
    #X_trainv, X_test = data_.loc[trainv_ix], data_.loc[test_ix]
    
    #good juncture to fork locally
    #for train_ix, valid_ix in tscv_inner.split(X_trainv):
    #for train_ix, valid_ix in cv_inner.split(X_trainv):
    X_valids = []
    
    for fold, (train_ix, valid_ix) in enumerate(tscv_inner.split(X_trainv)):
        #print("Fold: {}".format(fold))
        #print("TRAIN indices:", train_ix, "\n", "TEST indices:", valid_ix)
        #print("\n")
        X_train, X_valid = X_trainv.iloc[train_ix], X_trainv.iloc[valid_ix]
        
        X_valids.append(X_valid)
        
        #y_train, y_valid = y_trainv.iloc[train_ix], y_trainv.iloc[valid_ix]
        
        #print(X_train)
        if (model_==False):
            #print(np.shape(X_train))
            #model = pm.auto_arima(X_train, seasonal=True, m=4,stepwise=True)
            model = pm.auto_arima(X_train, seasonal=True, m=4)
            #model.fit()
            forecasts = model.predict(len(X_valid))  # predict N steps into the future
        else:
            #else:
            model = SARIMAX(X_train,order=model_.order,seasonal_order=model_.seasonal_order, intercept=model_.with_intercept)
            model.fit()
            #model.fit(disp=0)
            #forecasts = model.predict(start=len(X_train)+1,end=len(X_train)+len(X_valid))
            
            #model_fit = model.fit(start=0, end=len(X_train),exog=X_train,disp=False)
            model_fit = model.fit(disp=False)
            
            forecasts = model_fit.predict(start=len(model_fit.fittedvalues)+1, end=len(model_fit.fittedvalues)+len(X_valid),dynamic=True)
            #print(forecasts)
            #print(X_valid)
            #sm.tsa.statespace.SARIMAX(X_train, seasonal=True, m=4, order=my_order, seasonal_order=my_seasonal_order, ...)
            #pm.auto_arima(X_train, seasonal=True, m=4,)
            
        #plt.figure(figsize  = (8,8))
        #plt.title('Forecast result and Test data')
        # Visualize the forecasts (blue=train, green=forecasts)
        #x = np.arange(y.shape[0])
        #plt.plot(x[:150], train, c='blue')
        #plt.plot(x[150:], forecasts, c='green', label = 'ARIMA Forecast')
        #plt.plot(x[150:], test, c='red', label = 'Test Data')
        #plt.legend()
        #plt.show()
        
        inner_models.append(model)

        #yhat = model.predict(X_valid[t])
        yhat = forecasts
        # evaluate the model
        acc = np.mean(abs(X_valid.values.ravel()-forecasts))
        # store the result
        inner_results.append(acc)
        # report progress
        #print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
        # summarize the estimated performance of the model

    #good juncture to fork locally
    model_scores = list()

    #test inner model as ansemble
    for m_ in range(0,len(inner_models),1):
        m = inner_models[m_]
        X_valid = X_valids[m_]
        
        if(str(type(m))=="<class 'statsmodels.tsa.statespace.sarimax.SARIMAX'>"):
            
            #model_fit = m.fit(disp=0)
            model_fit = m.fit(disp=False)
            forecasts = model_fit.predict(start=len(model_fit.fittedvalues)+1,end=len(X_test)+len(X_valid))

            predict = model_fit.predict(start=(len(model_fit.fittedvalues)+len(X_valid)+1),end=(len(model_fit.fittedvalues)+len(X_valid)+len(X_test)))
            #print(predict)
            model_scores.append(np.mean(abs(predict)-X_test.values.ravel()))
        else:
            
            model_scores.append(np.mean(abs(m.predict(len(X_test))-X_test.values.ravel())))

    #average internal model results and report as outer_result which*
    return([np.mean(np.abs(model_scores)),inner_models])
    #outer_results.append(np.mean(np.abs(model_scores)))
    #outer_models.append(inner_models)

    #print('I Accuracy: %.3f (%.3f)' % (mean(inner_results), std(inner_results)))


In [ ]:
inner_arima(outer_sets[0])

In [ ]:
#model = ARIMA(data_, order=(0,0,0))
#model.fit()

In [ ]:
def evaluate_arima_model(X_train, X_test, arima_order):
    # prepare training dataset
    # make predictions
    history = [x for x in X_train.values]
    predictions = list()
    for t in range(0,len(X_test),1):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit()
        yhat = model_fit.forecast()[0]
        #print(yhat)
        predictions.append(yhat)
        history.append(X_test.values[t])
    # calculate out of sample error
    #SEM
    SEM = np.sqrt(np.sum(np.power((X_test.values - predictions),2))/(len(X_test)-1))
    #error = mean_squared_error(X_test, predictions)
    #print(SEM)
    return SEM

def evaluate_models(X_train, X_test, orders):
    best_score, best_cfg = float("inf"), None
    parameters = []
    for o in orders:
        order = o
   
    rmse = evaluate_arima_model(X_train, X_test, order)
    if rmse < best_score:
        best_score, best_cfg = rmse, order
    #print('ARIMA%s RMSE=%.3f' % (order,rmse))
    #print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))
    return([best_cfg, best_score])

[[(2, 0, 1), 7517.0647067634345],
 [(2, 0, 1), 8171.49011504315],
 [(2, 0, 1), 7229.464139381818],
 [(2, 0, 1), 6391.825279838172],
 [(2, 0, 1), 7083.866923979185],
 [(2, 0, 1), 7837.371912954673],
 [(2, 0, 1), 5640.745368676535],
 [(2, 0, 1), 6399.824664090937],
 [(2, 0, 1), 7882.76387574643],
 [(2, 0, 1), 6627.652901344845]]

In [ ]:
#outer_sets

In [46]:
cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
X = data_

warnings.simplefilter("ignore")
warnings.filterwarnings('ignore')

bestModels = []

outer_results = []

for train_ix, test_ix in cv_outer.split(X):

    X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
    
    tscv_inner = TimeSeriesSplit(n_splits=k_)
    
    X_trainv = X_train
   
    inner_results = list()
    inner_models = list()

    #for trainv_ix, test_ix in ts_cv_kfolds(data_.index,k_):
    
    # split data
    
    #X_trainv, X_test = data_.loc[trainv_ix], data_.loc[test_ix]
    
    orders = [
    (1,0,1),
    (0,1,0),
    (0,2,1),
    (0,2,3),
    (2,0,1)]
    
    intercepts = [True,False]
    
    #good juncture to fork locally
    #for train_ix, valid_ix in tscv_inner.split(X_trainv):
    #for train_ix, valid_ix in cv_inner.split(X_trainv):
    X_valids = []
    
    for fold, (train_ix, valid_ix) in enumerate(tscv_inner.split(X_trainv)):
        #print("Fold: {}".format(fold))
        X_train, X_valid = X_trainv.iloc[train_ix], X_trainv.iloc[valid_ix]
        
        X_valids.append(X_valid)
        
        cv_inner = KFold(n_splits=3, shuffle=True, random_state=1)

        bestModels.append(evaluate_models(X_train, X_valid, orders))
    
    #apply best model to test data
    best_order = stats.mode([b[0] for b in bestModels])[0][0]
    outer_results.append([best_order,evaluate_arima_model(X_valid,X_test,best_order)])

        
    #model_ = ARIMA(data_.values,order=stats.mode([b[0] for b in bestModels])[0][0])
    #model_fit = model_.fit()
    #forecasts = model_fit.predict(1,len(data_))


    
    """
    space = dict()
    space['seasonal'] = [True]
    space['n'] = [4] 
    
    model = ARIMA()
    search = GridSearchCV(model, space, scoring='neg_mean_squared_error', cv=cv_inner, refit=True)
    """
    
    """
    model = pm.auto_arima(X_train, seasonal=True, m=4)
    
    # execute search
    #result = model.fit(X_train, y_train)
    forecasts = model.predict(len(X_test))  # predict N steps into the future
    
    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_
    # evaluate model on the hold out dataset
    yhat = best_model.predict(X_test)
    # evaluate the model
    acc = accuracy_score(y_test, yhat)
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
    # summarize the estimated performance of the model
    print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))
    """

In [47]:
outer_results

[[array([2, 0, 1]), 7547.706154343495],
 [array([2, 0, 1]), 7812.891296911062],
 [array([2, 0, 1]), 7143.723166962982],
 [array([2, 0, 1]), 6285.849972029581],
 [array([2, 0, 1]), 6798.240955782283],
 [array([2, 0, 1]), 7657.907908933533],
 [array([2, 0, 1]), 5615.339181898318],
 [array([2, 0, 1]), 6419.842271960961],
 [array([2, 0, 1]), 7727.467464605026],
 [array([2, 0, 1]), 6595.929294322947]]

In [ ]:
#final_model = ARIMA(data_,order=stats.mode([b[0] for b in bestModels])[0][0])

model_ = ARIMA(data_.values,order=stats.mode([b[0] for b in bestModels])[0][0])
model_fit = model_.fit()
forecasts = model_fit.predict(1,len(data_))

#print(final_model)
#final_model.fit()
#final_model.predict(start=len(data_), end=len(data_), dynamic=True)

In [ ]:
pd.DataFrame(forecasts)

In [ ]:
data_

In [ ]:
plt.plot(pd.DataFrame(forecasts))
plt.show()
plt.plot(data_)
plt.show()

In [ ]:
#
final_model.fit(disp=0)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# fit model
model = ARIMA(data_, order=stats.mode([b[0] for b in bestModels])[0][0])
model_fit = model.fit()
# print(model_fit.summary())

# plot residual erros
residuals = pd.DataFrame(model_fit.resid)
#final_model.resid

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
plot_acf(residuals)
plt.show()
plot_pacf(residuals)
plt.show()

In [ ]:
"""
for o in range(0,len(outer_sets)):
    print(o)
    print(inner_arima(outer_sets[o]))
"""

In [ ]:

client = Client('192.168.3.100:8786')
#client.restart()

future = client.map(inner_arima, outer_sets)

results = client.gather(future)

In [ ]:
results

In [ ]:
np.mean([r[0] for r in results])

In [ ]:
results[0][1][0].seasonal_order

In [ ]:
results[0][1][0].order

In [ ]:
#dir(results[0][1][0])

In [ ]:
"""
client = Client('192.168.3.100:8786')
#client.restart()

future = client.map(evaluate_arima_model, outer_sets)

results = client.gather(future)
"""
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(data_, arima_order):
    
    k_=3
    #tscv_outer = TimeSeriesSplit(n_splits=k_,max_train_size=int(len(data_)/k_),test_size=int(len(data_)/k_))
    tscv_outer = TimeSeriesSplit(n_splits=k_)

    outer_results = []
    outer_models = []

    outer_sets = []

    for fold, (train_index, test_index) in enumerate(tscv_outer.split(data_)):
        print("Fold: {}".format(fold))
        print("TRAIN indices:", train_index, "\n", "TEST indices:", test_index)
        print("\n")
        X_train, X_test = data_.iloc[train_index], data_.iloc[test_index]
        outer_sets.append([X_train, X_test])
        
    for test_sets in outer_sets:
        #test_set = 0
        k_=2

        #cv_inner = KFold(n_splits=k_, shuffle=False)
        #tscv_inner = TimeSeriesSplit(n_splits=k_,max_train_size=int(len(test_sets[0])/k_),test_size=int(len(test_sets[0])/k_))    
        tscv_inner = TimeSeriesSplit(n_splits=k_)

        X_trainv = test_sets[0]
        #print(X_trainv)
        X_test = test_sets[1]
        #print(X_test)
        #print(X_test.index)

        inner_results = list()
        inner_models = list()

        x_valids = []
        
        for fold, (train_ix, valid_ix) in enumerate(tscv_inner.split(X_trainv)):
            
            X_train, X_valid = X_trainv.iloc[train_ix], X_trainv.iloc[valid_ix]
            x_valids.append(X_valid)
            
            history = [x for x in X_train]
            # make predictions
            predictions = list()
            for t in range(len(X_valid)):
                model = ARIMA(history, order=arima_order)
                model_fit = model.fit()
                yhat = model_fit.forecast()[0]
                predictions.append(yhat)
                history.append(test[t])
            # calculate out of sample error
            error = mean_squared_error(test, predictions)

            # evaluate the model
            acc = np.mean(abs(test-yhat))
            # store the result
            inner_results.append(acc)
            # report progress
            #print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
            # summarize the estimated performance of the model

        #good juncture to fork locally
        model_scores = list()

        #test inner model as ansemble
        for m_ in range(0,len(inner_models),1):
            m = inner_models[m_]
            X_valid = x_valids[m_]

            if(str(type(m))=="<class 'statsmodels.tsa.statespace.sarimax.SARIMAX'>"):

                #model_fit = m.fit(disp=0)
                model = ARIMA(history, order=arima_order)
                model_fit = m.fit(disp=False)
                forecasts = model_fit.predict(start=len(model_fit.fittedvalues)+1,end=len(X_test)+len(X_valid))

                predict = model_fit.predict(start=(len(model_fit.fittedvalues)+len(X_valid)+1),end=(len(model_fit.fittedvalues)+len(X_valid)+len(X_test)))
                #print(predict)
                model_scores.append(np.mean(abs(predict)-X_test.values.ravel()))
            
        #average internal model results and report as outer_result which*
    
    return(np.mean([r[0] for r in results]))
    
    return([np.mean(np.abs(model_scores)),inner_models])
    #outer_results.append(np.mean(np.abs(model_scores)))
    #outer_models.append(inner_models)

    #print('I Accuracy: %.3f (%.3f)' % (mean(inner_results), std(inner_results)))


In [ ]:
orders = [[(1,0,1)],
[(0,1,0)],
[(0,2,1)],
[(0,2,3)],
[(2,0,1)]]

intercepts = [True,False]

parameters = []
for o in orders:
    for i in intercepts:
        parameters.append([o,i])


In [ ]:

client = Client('192.168.3.100:8786')
#client.restart()

future = client.map(inner_arima, outer_sets)

results = client.gather(future)

In [ ]:
#forecasts = model_.predict(5)  # predict N steps into the future

In [ ]:
model_ = pm.auto_arima(data_, seasonal=True, m=4)
model = SARIMAX(data_,order=model_.order,seasonal_order=model_.seasonal_order, intercept=model_.with_intercept)
#model.fit(disp=0)
#forecasts = model.predict(start=len(X_train)+1,end=len(X_train)+len(X_valid))

model_fit = model.fit(start=0, end=len(data_),exog=data_,disp=False)
forecasts = model_fit.predict(start=len(data_)+1, end=len(data_)+10,dynamic=True)

In [ ]:
inner_arima(outer_sets[0],results[0][1][0])

In [ ]:
strings = []
for i in list(matplotlib.cbook.flatten([r[1] for r in results])):
    strings.append(str(i))

In [ ]:
strings.sort()

In [ ]:
[l.order for l in list(matplotlib.cbook.flatten([r[1] for r in results]))]
[l.seasonal_order for l in list(matplotlib.cbook.flatten([r[1] for r in results]))]
[l for l in list(matplotlib.cbook.flatten([r[1] for r in results]))]

In [ ]:
strings

In [ ]:
string_counts = []

for u in np.unique(strings):
    #print(u)
    count = np.count_nonzero(np.array(strings)==u)
    string_counts.append([u,count])
    #print(np.array(strings)==u)

In [ ]:
[s[0] for s in string_counts]

In [ ]:
sorted_string_counts_df = pd.DataFrame(string_counts,columns=['arima model','times occurred in nested cv']).sort_values(by='times occurred in nested cv',ascending=False)

In [ ]:
len(sorted_string_counts_df)

In [ ]:
sorted_string_counts_df.head(10)

In [ ]:
sorted_string_counts_df['arima model']

In [ ]:
#inner_arima(outer_sets[0],

In [ ]:
strings==np.unique(strings)